# SAT Formulation for Multiple Couriers Planning

## 1. Import Required Libraries and Utilities

In [1]:
from z3 import BoolVal, Bool, Implies, And
import numpy as np
from utils2 import *  
from encodings_utils import *
import time

## 2. Define the SAT Model for Multiple Couriers Planning

In [2]:
# timer
# optimality, 
# symmetry breaking, 
# cambiare le instanze con lower and upper bouns, 
# rivedere

In [3]:
def mcp_sat(m, n, l, s, D, search='linear', time_limit=300000, symm_constr=False): 
    
    start_time = time.time()
    solver = Solver()
    solver.set("random_seed", 42)

    deposit = n  
    max_load = sum(s)
    D_max = np.max(D)
    s_max = max(s)
    l_max = max(l)

    # Variables
    path = [[[Bool(f'p_{courier}_{package}_{step}') for step in range(n + 2)] for package in range(n + 1)] for courier in range(m)]
    courier_weights = [[Bool(f'w_{courier}_{package}') for package in range(n)] for courier in range(m)]
    courier_loads = [[Bool(f'cl_{courier}_{bit}') for bit in range(num_bits(max_load))] for courier in range(m)]
    c_dist_tot = [[Bool(f'cdt_{courier}_{bit}') for bit in range(num_bits(D_max * (n + 1)))] for courier in range(m)]
    max_dist_b = [Bool(f'max_d_{bit}') for bit in range(num_bits(D_max * (n + 1)))]
    c_dist_par = [[[Bool(f"cpt_{courier}_{step}_{bit}") for bit in range(num_bits(D_max))] for step in range(n+1)] for courier in range(m)]

    s_b = [int_to_bin(s_value, num_bits(s_max)) for s_value in s]
    l_b = [int_to_bin(l_value, num_bits(l_max)) for l_value in l]
    D_b = [[int_to_bin(D[i][j], num_bits(D_max)) for j in range(n+1)] for i in range(n+1)]
    


    # Constraints
    for courier in range(m):
        for step in range(n + 2):
            solver.add(exactly_one_np([path[courier][package][step] for package in range(n + 1)]))
 
    for package in range(n):
       solver.add(exactly_one_np([path[courier][package][step] for courier in range(m) for step in range(n + 2)]))
    
    
    for courier in range(m):
        solver.add(path[courier][deposit][0] == True)
        solver.add(path[courier][deposit][n + 1] == True)
        solver.add(cond_sum_bin(s_b, courier_weights[courier], courier_loads[courier],f"courier_load_{courier}"))
        solver.add(geq(l_b[courier],courier_loads[courier]))
        
        for step in range(n + 2):
            for package in range(n):
                solver.add(Implies(path[courier][package][step], courier_weights[courier][package]))


    # Distance calculations
    for courier in range(m):
        for step in range(n + 1):
            for package_start in range(n + 1):
                for package_end in range(n + 1):
                    solver.add(Implies(And(path[courier][package_start][step], path[courier][package_end][step+1]), 
                                  eq_bin(D_b[package_start][package_end],c_dist_par[courier][step])))                
        solver.add(cond_sum_bin(c_dist_par[courier], [BoolVal(True) for _ in range(n+1)], c_dist_tot[courier], f"def_courier_dist_{courier}"))
    solver.add(max_var(c_dist_tot, max_dist_b))


    '''
    #Symmetric breaking constraint
    if symm_constr == True:
        l_sorted = [(l[i],i) for i in range(len(l))]
        l_sorted.sort(key = lambda x: x[0], reverse= True)

        for i in range(m-1):
            if l_sorted[i][0] == l_sorted[i+1][0]:

                #S1 if the load of two couriers are equal then define the lexicographical order
                solver.add(lex_leq(courier_weights[l_sorted[i][1]], courier_weights[l_sorted[i+1][1]]))

            else:
                #S2 if the load of the first courier is greater than the load of the second courier then impose an order
                solver.add(geq(courier_loads[l_sorted[i][1]],courier_loads[l_sorted[i+1][1]]))
    '''
    
    # Linear or binary search
    if search == 'linear':
        satisfiable = True
        last_model = None
        upper_bound = D_max * n
        while satisfiable:
            remaining_time = max(0, int((time_limit - (time.time() - start_time)) * 1000))  
            solver.set("timeout", 0)
            solver.push()
            solver.add(geq(int_to_bin(upper_bound, num_bits(D_max * n)), max_dist_b))
            if solver.check() == sat:
                model = solver.model()
                last_model = model
                upper_bound = bin_to_int([model[b] for b in max_dist_b]) 
                print(f"Found solution with upper_bound: {upper_bound}")
            else:
                satisfiable = False
            solver.pop()
        if last_model is None:
            return time.time() - start_time, False, None, None
        obj_value = bin_to_int([last_model[b] for b in max_dist_b])
        print(f"Final objective value: {obj_value}")
        return time.time() - start_time, True, obj_value, last_model
    
    elif search == 'binary':
        lower_bound, upper_bound = 0, D_max * n
        last_model = None
        while upper_bound - lower_bound > 1:
            remaining_time = max(0, int((time_limit - (time.time() - start_time)) * 1000))  
            solver.set("timeout", remaining_time)
            mid = (upper_bound + lower_bound) // 2
            solver.push()
            solver.add(geq(int_to_bin(mid, num_bits(D_max * n)), max_dist_b))
            if solver.check() == sat:
                model = solver.model()
                last_model = model
                upper_bound = bin_to_int([model[b] for b in max_dist_b])
                print(f"Upper_bound: {upper_bound}")
            else:
                lower_bound = mid
            solver.pop()
        if last_model is None:
            return time.time() - start_time, False, None, None, search
        obj_value = bin_to_int([last_model[b] for b in max_dist_b])
        print(f"Final objective value: {obj_value}")
        return time.time() - start_time, True, obj_value, last_model, search

## 3. Example Usage

In [7]:
instance_num = "01"
file_path = os.path.join('Instances', f'inst{instance_num}.dat')
instance = read_dat_file(file_path)
m = instance["m"]  
n = instance["n"]  
l = instance["l"]  
s = instance["s"]
D = instance["D"]  

time_taken, opt, obj_value, solution, search = mcp_sat(m, n, l, s, D, search='binary', symm_constr=False)
if solution:
    depot = n
    solution = refineSol(time_taken, opt, obj_value, solution, search, depot)
    print('Solution:', solution)
else:
    print('No solution found')

Upper_bound: 16
Upper_bound: 14
Final objective value: 14
Solution: {'binary': {'time': 0, 'optimal': True, 'obj': 14, 'sol': [[4, 3, 1], [2, 5, 6]]}}
